In [1]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import torch
import numpy as np
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict
from PIL import Image
from matplotlib.ticker import FormatStrFormatter
from copy import copy
import os
import shutil
import csv

In [2]:
def load_model(path, model):
    checkpoint = torch.load(path)
    m_name = checkpoint["model_name"]
    m_sd = checkpoint["state_dict"]
    m_classifier = checkpoint["classifier"]
    if m_name == "resnet50":
        model.fc = m_classifier
    else:
        model.classifier = m_classifier
    model.load_state_dict(m_sd)
    model.class_to_idx = checkpoint["class_to_idx"]
    return model

In [3]:
def getClassDict():
    data_dir = "C:/Users/Alamjeet Singh/Desktop/KGExtract/train"
    train_transforms = transforms.Compose([transforms.Resize((224,224)),
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485, 0.456, 0.406], 
                                                                [0.229, 0.224, 0.225])])
    full_data = datasets.ImageFolder(data_dir, transform = train_transforms)
    cldict = full_data.class_to_idx
    for cl, probs in cldict.items():
        cldict[cl] = 0
    return cldict

In [4]:
cldict = getClassDict()

In [5]:
M9664_10000= models.densenet161(pretrained=True)
M13_17     = models.resnet50(pretrained=True)
M22_30     = models.resnet50(pretrained=True)
M34_39     = models.resnet50(pretrained=True)
M50_52     = models.resnet50(pretrained=True)
M65_78     = models.resnet50(pretrained=True)
M91_117    = models.resnet50(pretrained=True)
M130_169   = models.resnet50(pretrained=True)
M182_247   = models.resnet50(pretrained=True)
M260_325   = models.resnet50(pretrained=True)
M351_455   = models.resnet50(pretrained=True)
M468_611   = models.resnet50(pretrained=True)
M624_741   = models.resnet50(pretrained=True)
M793_949   = models.resnet50(pretrained=True)

C:\AnaconML\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [8]:
M9664_10000= load_model("KG9664-10000_1.pth", M9664_10000)
M13_17     = load_model("KG13-17_1.pth",M13_17)
M22_30     = load_model("KG22-30_1.pth",M22_30)
M34_39     = load_model("KG34-39_1.pth",M34_39)
M50_52     = load_model("KG50-52_1.pth",M50_52)
M65_78     = load_model("KG65-78_1.pth",M65_78)
M91_117    = load_model("KG91-117_1.pth",M91_117)
M130_169   = load_model("KG130-169_1.pth",M130_169)
M182_247   = load_model("KG182-247_1.pth",M182_247)
M260_325   = load_model("KG260-325_1.pth",M260_325)
M351_455   = load_model("KG351-455_1.pth",M351_455)
M468_611   = load_model("KG468-611_1.pth",M468_611)
M624_741   = load_model("KG624-741_1.pth", M624_741)
M793_949   = load_model("KG793-949_1.pth", M793_949)

In [9]:
torch.set_default_tensor_type("torch.cuda.FloatTensor")
#testdir = "C:/Users/Alamjeet Singh/Desktop/KGExtract/test"

In [12]:
m = {M13_17:2076,M22_30:1287,M34_39:569,M50_52:274,M65_78:308,M91_117:224,M130_169:125,M182_247:79,
     M260_325:31,M351_455:12,M468_611:10,M624_741:5,M793_949:4,M9664_10000:2}

In [13]:
def predict2(image_path, marr,dd):
    img = Image.open(image_path)
    img = img.resize((224,224))
    img = img.convert("RGB")
    trans2 = transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    trans1 = transforms.ToTensor()
    x = trans1(img)
    x = trans2(x)
    x = x.type(torch.cuda.FloatTensor).unsqueeze(0)
    for model,numclass in marr.items():
        model.eval()
        model.to("cuda")
        topk = numclass
        output = model.forward(x)
        ps = torch.exp(output).data.topk(topk)
        ps_tf = ps[0].cpu()
        classes = ps[1].cpu()
        class_to_idx_i = {model.class_to_idx[i]: i for i in model.class_to_idx}
        cls = [class_to_idx_i[label] for label in classes.numpy()[0]]
        p = ps_tf.numpy()[0]
        for i in range(len(p)):
            if cls[i] != "other":
                dd[cls[i]] = p[i]
        return dd

In [ ]:
#test_path = "C:/Users/Alamjeet Singh/Desktop/KGExtract/test/000dcf7d8.jpg"
#ret = ensemblePredict(test_path, m)

In [ ]:
ret

In [22]:
def ensemblePredict(path, modelarr):
    dd= getClassDict()
    dd = predict2(path,modelarr,dd)
    clarr = []
    parr = []
    for clname, probs in dd.items():
        clarr.append(clname)
        parr.append(probs)
    parrt = torch.tensor(parr).data.topk(5)
    pp = np.array(parrt[0].cpu())
    ppi = np.array(parrt[1].cpu())
    fclass = []
    for index in ppi:
        fclass.append(clarr[index])
    return fclass

In [23]:
import pandas as pd
df = pd.read_csv("C:/Users/Alamjeet Singh/Downloads/leaks.csv")
leakname = np.array(df["b_test_img"])
leaklabel = np.array(df["b_label"])

In [24]:
d = {}
for i in range(len(leakname)):
    d[leakname[i]] = leaklabel[i]

In [25]:
def writetoCSV(img_name, prediction_array):
    with open("C:/Users/Alamjeet Singh/Desktop/KGSubmissions/K36_14Ensemble.csv", mode = "a", newline='') as test_file:
        w = csv.writer(test_file, quoting=csv.QUOTE_NONE, delimiter = ",")
        y = [img_name]
        for element in prediction_array:
            y.append(element)
        w.writerow(y)

In [37]:
dfd = pd.read_csv("C:/Users/Alamjeet Singh/Desktop/KGSubmissions/K36_14Ensemble.csv")

In [38]:
dfd = np.array(dfd["00028a005.jpg"])

In [39]:
dfd = np.append(dfd,"00028a005.jpg")

In [40]:
len(dfd)

4263

In [41]:
files = os.listdir("C:/Users/Alamjeet Singh/Desktop/KGExtract/test")
i = 1
for file in files:
    if file not in dfd:
        x = ensemblePredict(str("C:/Users/Alamjeet Singh/Desktop/KGExtract/test/"+file), m)
        y = ""
        if file in leakname:
            y = " "+d[file]
        for element in x:
            y = y+" "+element
        x = [y]
        writetoCSV(file,x)
        print("{} written".format(str(i)+"/"+str(len(files))))
        i+=1

1/7960 written
2/7960 written
3/7960 written
4/7960 written
5/7960 written
6/7960 written
7/7960 written
8/7960 written
9/7960 written
10/7960 written
11/7960 written
12/7960 written
13/7960 written
14/7960 written
15/7960 written
16/7960 written
17/7960 written
18/7960 written
19/7960 written
20/7960 written
21/7960 written
22/7960 written
23/7960 written
24/7960 written
25/7960 written
26/7960 written
27/7960 written
28/7960 written
29/7960 written
30/7960 written
31/7960 written
32/7960 written
33/7960 written
34/7960 written
35/7960 written
36/7960 written
37/7960 written
38/7960 written
39/7960 written
40/7960 written
41/7960 written
42/7960 written
43/7960 written
44/7960 written
45/7960 written
46/7960 written
47/7960 written
48/7960 written
49/7960 written
50/7960 written
51/7960 written
52/7960 written
53/7960 written
54/7960 written
55/7960 written
56/7960 written
57/7960 written
58/7960 written
59/7960 written
60/7960 written
61/7960 written
62/7960 written
63/7960 written
6

489/7960 written
490/7960 written
491/7960 written
492/7960 written
493/7960 written
494/7960 written
495/7960 written
496/7960 written
497/7960 written
498/7960 written
499/7960 written
500/7960 written


KeyboardInterrupt: 